In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import h5py
from pathlib import Path

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import matplotlib.gridspec as gridspec
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import colors
import seaborn as sns

import flammkuchen as fl
from scipy.interpolate import interp1d

C:\Users\admin\AppData\Local\Temp\ipykernel_27180\1709915033.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:

from megabouts.tracking_data import TrackingConfig, FullTrackingData, load_example_data
from megabouts.pipeline import FullTrackingPipeline
from megabouts.utils import (
    bouts_category_name,
    bouts_category_name_short,
    bouts_category_color,
    cmp_bouts,
)

In [4]:
from mb_helper import compute_angle_between_vect_tail, compute_angle_between_vect, exptrapolate_segments, mid_head, midpoint
from mb_helper import tail_angles, fin_preprocess, calculate_angles
from mb_helper import labels_cat, color

In [5]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


# Load Bouts

In [6]:
master_path = Path(Path(r'\\portulab.synology.me\data\Kata\Figures_Paper\Koetter-et-al_data\Testdata') / 'Raw_Data')
out_path = Path(master_path/'Processed_Data')
fish_paths = list(master_path.glob('*f[0-9]*'))
fish_paths

[WindowsPath('//portulab.synology.me/data/Kata/Figures_Paper/Koetter-et-al_data/Testdata/Raw_Data/240423_f0')]

In [7]:
fish= 0
fish_path = fish_paths[fish]
fish_id =  fish_paths[fish].name#[:-13]
exp_name = Path(fish_paths[fish]).parts[-2]
exp_name = 'testfish'
exp_name, fish_id

('testfish', '240423_f0')

### Load DLC

In [8]:
### Load and replace eye data
#t, left eye [0], right eye [1], endpoints of eye vector end points (point1xy, point2xy) for anterior and posterior
eye_coords = fl.load(fish_path/'eye_coords.h5')['eye_coords']
eye_coords.shape

(95423, 2, 2, 2)

In [9]:
left_eye_points = np.asarray(eye_coords)[:,0]
right_eye_points = np.asarray(eye_coords)[:,1]

l_anterior = np.asarray(left_eye_points[:,0])
l_posterior = np.asarray(left_eye_points[:,1])

left_eye_anterior_x = []
left_eye_anterior_y = []
left_eye_posterior_x = []
left_eye_posterior_y = []
for i in range(l_anterior.shape[0]):
    left_eye_anterior_x.append(l_anterior[i][0])
    left_eye_anterior_y.append(l_anterior[i][1])
    left_eye_posterior_x.append(l_posterior[i][0])
    left_eye_posterior_y.append(l_posterior[i][1])
    
r_anterior = np.asarray(right_eye_points[:,0])
r_posterior = np.asarray(right_eye_points[:,1])

right_eye_anterior_x = []
right_eye_anterior_y = []
right_eye_posterior_x = []
right_eye_posterior_y = []
for i in range(r_anterior.shape[0]):
    right_eye_anterior_x.append(r_anterior[i][0])
    right_eye_anterior_y.append(r_anterior[i][1])
    right_eye_posterior_x.append(r_posterior[i][0])
    right_eye_posterior_y.append(r_posterior[i][1])
    

## Load DLC

In [10]:
fps=200
mm_per_unit = 1/70
N_seg = 10
N = eye_coords.shape[0]

In [30]:

df = pd.read_csv(out_path/ '{}_DLC.csv'.format(fish_id), header=[0,1])
print(f'{df.shape[0]/(fps*60)} minutes at {fps} fps')
print('working on {} frames'.format(df.shape[0]))

#Extract angles
body_x = df.body.values[:, 0].astype('float')
body_y = df.body.values[:, 1].astype('float')

tail_x_col = [f'tail_{i}' for i in range(5)]
tail_y_col = [f'tail_{i}' for i in range(5)]
tail_x = np.array([df[x].iloc[:, 0].values.astype('float') for x in tail_x_col])
tail_y = np.array([df[x].iloc[:, 1].values.astype('float') for x in tail_y_col])

# upsample tail to 10 segments
tail_x_10, tail_y_10 = exptrapolate_segments(tail_x, tail_y, N_seg)


7.9519166666666665 minutes at 200 fps
working on 95423 frames


In [31]:
tail_x_10.shape

(11, 95423)

In [32]:
## replace eyes and recompute mid head

In [33]:
mid_headx, mid_heady, left_mid_eye_x, left_mid_eye_y, right_mid_eye_x, right_mid_eye_y = mid_head(
             np.asarray(right_eye_posterior_x), np.asarray(right_eye_posterior_y),
             np.asarray(right_eye_anterior_x), np.asarray(right_eye_anterior_y),
             np.asarray(left_eye_posterior_x), np.asarray(left_eye_posterior_y),
             np.asarray(left_eye_anterior_x), np.asarray(left_eye_anterior_y))


In [35]:
head_x.shape, head_y.shape

((95423,), (95423,))

#### Add head to df

In [36]:
# Now, let's add 'head_x' and 'head_y' keypoints under 'head' with the same likelihood as 'body'
for i in range(11):
    df[('tail_{}'.format(i), 'x')] = tail_x_10[i,:] 
    df[('tail_{}'.format(i), 'y')] = tail_y_10[i,:]
    df[('tail_{}'.format(i), 'likelihood')] = df[('body', 'likelihood')] # Copy likelihood of body to head

In [37]:
df[('left_eye_anterior', 'x')] = left_eye_anterior_x 
df[('left_eye_anterior', 'y')] = left_eye_anterior_y 

df[('left_eye_posterior', 'x')] = left_eye_posterior_x 
df[('left_eye_posterior', 'y')] = left_eye_posterior_y 

df[('right_eye_anterior', 'x')] = right_eye_anterior_x 
df[('right_eye_anterior', 'y')] = right_eye_anterior_y 

df[('right_eye_posterior', 'x')] = right_eye_posterior_x 
df[('right_eye_posterior', 'y')] = right_eye_posterior_y 

In [38]:
# Now, let's add 'head_x' and 'head_y' keypoints under 'head' with the same likelihood as 'body'
df[('mid_head', 'x')] = mid_headx 
df[('mid_head', 'y')] = mid_heady 
df[('mid_head', 'likelihood')] = df[('body', 'likelihood')] # Copy likelihood of body to head


In [39]:
df.head()

bodyparts DLC_filter     motor                     edge_filter  \
     coords        pos         z         y         x          ok   
0         0          1  0.000000  8.993996 -7.050400        True   
1         1          1 -0.000137  9.084572 -7.089576        True   
2         2          1  0.000000  9.179195 -7.108448        True   
3         3          1  0.000000  9.258852 -7.098520        True   
4         4          1 -0.000013  9.320093 -7.081586        True   

  left_eye_anterior                        left_eye_posterior  ...     tail_8  \
                  x           y likelihood                  x  ... likelihood   
0        645.326900  521.443017   0.999891         654.856002  ...   0.998467   
1        646.183150  519.644351   0.999896         656.078755  ...   0.998625   
2        650.207855  517.061760   0.999923         659.556388  ...   0.999190   
3        654.625610  513.145553   0.999913         664.477157  ...   0.997801   
4        657.924704  510.390997   0.999740         667.236973  ...   0.998626   

       tail_9                            tail_10                         \
            x           y likelihood           x           y likelihood   
0  463.398844  250.457989   0.998467  439.471039  239.365433   0.998467   
1  473.571332  246.601053   0.998625  444.348114  245.641525   0.998625   
2  491.481593  238.960160   0.999190  461.540009  236.261642   0.999190   
3  503.204287  227.416002   0.997801  476.081390  216.947006   0.997801   
4  506.768862  217.844257   0.998626  485.563690  199.774689   0.998626   

     mid_head                         
            x           y likelihood  
0  633.807176  544.905587   0.998467  
1  634.773111  542.948934   0.998625  
2  638.426775  540.509511   0.999190  
3  643.096984  536.308950   0.997801  
4  646.209680  533.732044   0.998626  

[5 rows x 70 columns]

In [40]:
df.columns

MultiIndex([(          'bodyparts',              'coords'),
            (         'DLC_filter',                 'pos'),
            (              'motor',                   'z'),
            (              'motor',                   'y'),
            (              'motor',                   'x'),
            (        'edge_filter',                  'ok'),
            (  'left_eye_anterior',                   'x'),
            (  'left_eye_anterior',                   'y'),
            (  'left_eye_anterior',          'likelihood'),
            ( 'left_eye_posterior',                   'x'),
            ( 'left_eye_posterior',                   'y'),
            ( 'left_eye_posterior',          'likelihood'),
            ( 'right_eye_anterior',                   'x'),
            ( 'right_eye_anterior',                   'y'),
            ( 'right_eye_anterior',          'likelihood'),
            ('right_eye_posterior',                   'x'),
            ('right_eye_posterior',     

In [41]:
df.to_csv(out_path/ '{}_DLC_mod.csv'.format(fish_id))


In [42]:
df_dlc = pd.read_csv(out_path/ '{}_DLC_mod.csv'.format(fish_id), header=[0, 1])


In [43]:
df_dlc.head()

Unnamed: 0_level_0 bodyparts DLC_filter     motor                      \
  Unnamed: 0_level_1    coords        pos         z         y         x   
0                  0         0          1  0.000000  8.993996 -7.050400   
1                  1         1          1 -0.000137  9.084572 -7.089576   
2                  2         2          1  0.000000  9.179195 -7.108448   
3                  3         3          1  0.000000  9.258852 -7.098520   
4                  4         4          1 -0.000013  9.320093 -7.081586   

  edge_filter left_eye_anterior                         ...     tail_8  \
           ok                 x           y likelihood  ... likelihood   
0        True        645.326900  521.443017   0.999891  ...   0.998467   
1        True        646.183150  519.644351   0.999896  ...   0.998625   
2        True        650.207855  517.061760   0.999923  ...   0.999190   
3        True        654.625610  513.145553   0.999913  ...   0.997801   
4        True        657.924704  510.390997   0.999740  ...   0.998626   

       tail_9                            tail_10                         \
            x           y likelihood           x           y likelihood   
0  463.398844  250.457989   0.998467  439.471039  239.365433   0.998467   
1  473.571332  246.601053   0.998625  444.348114  245.641525   0.998625   
2  491.481593  238.960160   0.999190  461.540009  236.261642   0.999190   
3  503.204287  227.416002   0.997801  476.081390  216.947006   0.997801   
4  506.768862  217.844257   0.998626  485.563690  199.774689   0.998626   

     mid_head                         
            x           y likelihood  
0  633.807176  544.905587   0.998467  
1  634.773111  542.948934   0.998625  
2  638.426775  540.509511   0.999190  
3  643.096984  536.308950   0.997801  
4  646.209680  533.732044   0.998626  

[5 rows x 71 columns]